# Resolução desafio Coodesh
Utilização linguagem Python, utilizando Pandas

In [1]:
from datetime import datetime
import pandas as pd

## Atributos dos datasets

In [3]:
dfAtributos = pd.read_csv("./analytics/dataset_info.csv")
dfAtributos

,seq,name,description
0,1,show_id,Unique ID for every Movie / Tv Show
1,2,type,Identifier - A Movie or TV Show
2,3,title,Title of the Movie / Tv Show
3,4,director,Director of the Movie
4,5,cast,Actors involved in the movie / show
5,6,country,Country where the movie / show was produced
6,7,data_added,Date it was added on Netflix or Amazon Prime
7,8,release_year,Actual Release year of the move / show
8,9,rating,TV Rating of the movie / show
9,10,duration,Total Duration - in minutes or number of seasons


(*) Ao comparar o arquivo attributes.txt com o dataset de filmes, foi possível identificar que faltavam colunas no arquivo de atributos. Foi feita a identificação dos campos e atualizada a documentação.

## Carregar os arquivos

In [ ]:
dfFilmesNetflix = pd.read_csv("./data/netflix_titles.csv")
dfFilmesNetflix.info()

In [ ]:
dfFilmesAmazon = pd.read_csv("./data/amazon_prime_titles.csv")
dfFilmesAmazon.info()

# Gerar um dataset unificado de filmes das duas plataformas

In [ ]:
# Como o campo show_id se repete em cada plataforma, 
# será necessário garantir que exista um código identificador único para cada dataset
# Para não perder a referência da plataforma, é necessário criar uma coluna com esta informação em cada dataframe

dfFilmesNetflix["title_id"] = "nfx_" + dfFilmesNetflix["show_id"]
dfFilmesNetflix["platform"] = "netflix"

dfFilmesAmazon["title_id"] = "amz_" + dfFilmesAmazon["show_id"]
dfFilmesAmazon["platform"] = "amazon_prime" 

dfFilmesNetflix.info()

In [ ]:
dfFilmesAmazon.info()

In [ ]:
# Criar base de filmes unificada, contendo os filmes das duas plataformas
dfFilmes = pd.concat([dfFilmesNetflix, dfFilmesAmazon],ignore_index=True)
dfFilmes.info()

## Fazer o processamento do dataset unificado

Para modelagem dos dados, é necessário transformar as colunas que o conteúdo representa uma lista, em um arquivos de dados que conterá o relacionamento com o identificador do título.

* director -> directors_title.csv
* cast -> casting_title.csv
* listed_in -> categories_title.csv
* country -> countries_title.csv

Para facilitar o trabalho com data, o campo date_added precisa ser convertido para o formato YYYY-MM-DD. 

In [ ]:
# Função para extrair os valores dos campos que apresentam lista e relacionar ao id do título
def extrairListaDeRelacionamento(df:pd.DataFrame, coluna:str, camposExtras=[]):
    lista = []
    for index,linha in df.iterrows():
        if not pd.isna(linha[coluna]):
            id = linha["title_id"]
            itens = str(linha[coluna]).split(",")
            for item in itens:
                lista.append([id,item.strip()])
    return lista

In [ ]:
dfFilmes[dfFilmes['title_id'].duplicated() == True].info()

In [ ]:
listaDiretores = extrairListaDeRelacionamento(dfFilmes,"director")
dfDirector = pd.DataFrame(listaDiretores,columns = ['title_id', 'director'])
dfDirector.to_csv("./analytics/directors_title.csv",index=False)

dfDirector.describe()

In [ ]:
listaArtistas = extrairListaDeRelacionamento(dfFilmes,"cast")
dfCasting = pd.DataFrame(listaArtistas,columns = ['title_id', 'artist'])
dfCasting.to_csv("./analytics/casting_title.csv",index=False)

dfCasting.describe()

In [ ]:
listaPaises = extrairListaDeRelacionamento(dfFilmes,"country")
dfCountries = pd.DataFrame(listaPaises,columns = ['title_id', 'country'])
# Incluir no dataset dos países a plataforma
dfCountries = dfCountries.merge(
    dfFilmes.rename({'country': 'country_list'}, axis=1),
    on='title_id',
    how='left')[["title_id","country","platform"]]

dfCountries.to_csv("./analytics/countries_title.csv",index=False)
dfCountries.describe()

In [ ]:
listaCategorias = extrairListaDeRelacionamento(dfFilmes,"listed_in")
dfCategories = pd.DataFrame(listaCategorias,columns = ['title_id', 'category'])
dfCategories.to_csv("./analytics/categories_title.csv",index=False)

dfCategories.describe()

In [ ]:
def converterData(data:str):
    try:
        mesDia,ano = data.split(", ")
        mes,dia = mesDia.split(" ")
        dataTexto =  f"{mes} {dia.zfill(2)}, {ano}"
        novaData = datetime.strptime(dataTexto,"%B %d, %Y")
        return novaData.strftime("%Y-%m-%d")
    except Exception as e:
        pass

In [ ]:
dfFilmes["date_added"] = dfFilmes["date_added"].apply(converterData)
dfFilmes.describe()

In [ ]:
#Excluir colunas de lista
novoDfFilmes = dfFilmes.drop(columns=["director","cast","country","listed_in"])
#Salvar novo dataset de filmes unificado
novoDfFilmes.to_csv("./analytics/titles.csv",index=False)